In [1]:
import sys 
sys.path.insert(0,"../../../../utilities/")
import data_utils
sys.path.insert(0,"../../src/")
import util, mrf
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

In [2]:
resolution = 25000
input_file = "../../data/test_input_chr1.txt"
bin_file = "../../data/test_bin_chr1.bed"
hic_file = "../../data/test_Hi-C_chr1.txt"

In [3]:
input_data = util.readData(input_file)
bin_data = util.readBedGraph(bin_file)
(n, d) = input_data.shape
edges = util.create_hic_matrix(hic_file, n)

In [37]:
boundaries = []
lens = []
for i in range(len(bin_data)-1):    
    boundaries.append(bin_data[i][2] != bin_data[i+1][1])
    lens.append((bin_data[i][2]-bin_data[i][1])!=25000)

In [51]:
edge_data = []
edge_data.append([0,1,1])
for i in range(input_data.shape[0]-2):
    edge_data.append([i+1,i,1])
    edge_data.append([i+1,i+2,1])
edge_data.append([input_data.shape[0]-1,input_data.shape[0]-2,1])
edge_data = np.array(edge_data)

In [15]:
#### GMM ####

from sklearn import mixture
gmm_model = mixture.GaussianMixture(n_components=10, covariance_type='full')
gmm_model.fit(input_data)
gmm_labels = gmm_model.predict(input_data)

In [16]:
#### GMM+HMM ####

from hmmlearn import hmm
gmm_hmm_model = hmm.GaussianHMM(n_components=10, covariance_type='full')
gmm_hmm_model.fit(input_data)
gmm_hmm_labels = gmm_hmm_model.decode(input_data)

In [48]:
np.unique(gmm_labels, return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 303, 1023, 1100,  977,  630, 1250,  961, 1842,  158,  994]))

In [50]:
np.unique(gmm_hmm_labels[1], return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32),
 array([ 941, 1726,  952, 1281,  706,  819,  584,  696,  696,  837]))

In [10]:
labels_df = 
print(len(MAP_estimate[1]), len(label))

9238 9238


In [18]:
labels_df = pd.DataFrame(bin_data)
labels_df['GMM_labels'] = gmm_labels
labels_df['GMM_HMM_labels'] = gmm_hmm_labels[1]

In [20]:
a = labels_df.groupby(['GMM_labels','GMM_HMM_labels']).size()
a_table = a.unstack(level=0)

In [46]:
a_table

GMM_labels,0,1,2,3,4,5,6,7,8,9
GMM_HMM_labels,,,,,,,,,,
0,NaN,NaN,79.0,73.0,NaN,3.0,786.0,NaN,NaN,NaN
1,NaN,29.0,NaN,NaN,NaN,NaN,NaN,1697.0,NaN,NaN
2,NaN,596.0,2.0,NaN,340.0,NaN,NaN,14.0,NaN,NaN
3,293.0,NaN,NaN,359.0,NaN,8.0,NaN,NaN,NaN,621.0
4,1.0,NaN,8.0,NaN,91.0,599.0,NaN,NaN,7.0,NaN
5,NaN,NaN,631.0,NaN,24.0,24.0,140.0,NaN,NaN,NaN
6,9.0,NaN,1.0,1.0,7.0,337.0,NaN,NaN,NaN,229.0
7,NaN,387.0,174.0,NaN,4.0,NaN,NaN,131.0,NaN,NaN
8,NaN,11.0,204.0,NaN,164.0,166.0,NaN,NaN,151.0,NaN


In [44]:
labels_map = {}
for l in range(len(a_table)):
    labels_map[l] = np.argmax(a_table[l])

In [45]:
labels_map

{0: 3, 1: 2, 2: 5, 3: 9, 4: 2, 5: 4, 6: 0, 7: 1, 8: 8, 9: 3}

In [52]:
#### GMM+chain HMRF ####

class Args:
  i = "../../data/test_input_chr1.txt"
  hic = "../../data/test_Hi-C_chr1.txt"
  w = 25000
  n = 10
  o = "results"
  g = "../../data/test_bin_chr1.bed"
args=Args()
hmrf = mrf.MarkovRandomField(n=n, edges=edge_data, obs=input_data, args=args)
hmrf.init_gmm()
hmrf.init_trans()
hmrf.solve()


label frequencies at iteration 0 is (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([ 763,  527, 1258,  655, 1018,  985, 1121, 1378,  443, 1090]))
label frequencies at iteration 1 is (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([ 708,  588,  879,  594,  863, 1345, 1141, 1383,  570, 1167]))
label frequencies at iteration 2 is (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([ 750,  616,  752,  674,  850, 1525,  991, 1367,  289, 1424]))
label frequencies at iteration 3 is (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([ 820,  573,  743,  682,  891, 1562,  755, 1382,  240, 1590]))
label frequencies at iteration 4 is (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([ 850,  444,  769,  699,  881, 1580,  693, 1413,  218, 1691]))
label frequencies at iteration 5 is (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([ 862,  340,  760,  750,  860, 1611,  687, 1431,  213, 1724]))
label frequencies at iteration 6 is (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([ 898,  223,  697,  760,  864, 1654,  634, 1451,

In [15]:
np.unique(label, return_counts=True)
print((label[1:]!=label[:-1]).sum())

912


In [17]:
np.unique(MAP_estimate[1], return_counts=True)
print((MAP_estimate[1][1:]!=MAP_estimate[1][:-1]).sum())

563


In [43]:
from scipy.special import logsumexp
logsumexp(hmrf.edge_potential)

-2.220446049250313e-16

In [15]:
fithic_data = pd.read_csv("../../../../data/K562/Hi-C/fithic_25000/chr1_chr1.txt", sep = "\t")

In [16]:
fithic_data.columns

Index(['chr1', 'fragmentMid1', 'chr2', 'fragmentMid2', 'contactCount',
       'p-value', 'q-value', 'bias1', 'bias2', 'ExpCC'],
      dtype='object')

In [19]:
fithic_significants = fithic_data.nsmallest(20000,'q-value')

In [20]:
lamin_tsa = pd.read_csv("../../../../data/K562/genomic-assays/TSA-seq/LaminA_TSA-seq_binned25000.bedgraph", sep = "\t", header = None)

In [22]:
lamin_tsa[lamin_tsa.iloc[:,0]=='chr1']

,0,1,2,3
0,chr1,0,1,-0.600856
1,chr1,1,2,-0.897333
2,chr1,2,3,-1.813977
3,chr1,3,4,-1.221924
4,chr1,4,5,-1.475075
5,chr1,5,6,-0.844969
6,chr1,6,7,-1.230400
7,chr1,7,8,-0.477270
8,chr1,8,9,-0.312351
9,chr1,9,10,-1.215879
